In [1]:
pip install PyQt5 mysql-connector-python

     --------------------------------------- 14.2/14.2 MB 11.3 MB/s eta 0:00:00
     ------------------------------------- 527.0/527.0 kB 11.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QLineEdit, QTextEdit, QDialog, QDialogButtonBox, QHBoxLayout, QMessageBox, QFormLayout, QInputDialog
from PyQt5.QtCore import Qt

import mysql.connector
from datetime import datetime

class SeedApp(QWidget):
    def __init__(self):
        super().__init__()

        # Check database connection on startup
        if not self.test_database_connection():
            self.show_error_message("Error: Unable to connect to the database. Please check your database connection settings.")
            sys.exit()

        # Show login dialog
        login_dialog = LoginDialog()
        if login_dialog.exec_() == QDialog.Accepted:
            self.institute_code = login_dialog.get_institute_code()
            self.institute_id = self.get_institute_id()
            self.init_ui()
            self.show()
        else:
            sys.exit()

    def init_ui(self):
        self.setWindowTitle('Seed Database Application')
        self.setGeometry(300, 300, 400, 300)

        self.label = QLabel('Seed Name:', self)
        self.seed_name_input = QLineEdit(self)

        self.search_button = QPushButton('Search Seed', self)
        self.search_button.clicked.connect(self.search_seed)

        self.retrieve_button = QPushButton('Retrieve Seed', self)
        self.retrieve_button.clicked.connect(self.retrieve_seed)

        self.export_button = QPushButton('Deposit Seed', self)
        self.export_button.clicked.connect(self.deposit_seed)

        self.view_species_button = QPushButton('View Seed Holder', self)
        self.view_species_button.clicked.connect(self.show_view_species_dialog)

        self.insert_seed_button = QPushButton('Insert Seed', self)
        self.insert_seed_button.clicked.connect(self.show_insert_seed_dialog)

        self.result_text = QTextEdit(self)

        # 레이아웃 구성
        layout = QVBoxLayout()

        layout.addWidget(self.label)
        layout.addWidget(self.seed_name_input)
        layout.addWidget(self.search_button)
        layout.addWidget(self.retrieve_button)
        layout.addWidget(self.export_button)
        layout.addWidget(self.result_text)
        layout.addWidget(self.view_species_button)
        layout.addWidget(self.result_text)
        layout.addWidget(self.insert_seed_button)

        self.setLayout(layout)
        

        # Check database connection on startup
        if not self.test_database_connection():
            self.show_error_message("Error: Unable to connect to the database. Please check your database connection settings.")
            sys.exit()

        self.show()

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Escape:
            self.close()

    def search_seed(self):
        seed_name = self.seed_name_input.text()
        seed_data = self.query_database(f"SELECT * FROM SEED WHERE SeedName = '{seed_name}'")
        if seed_data:
            self.display_result(seed_data)
        else:
            self.show_error_message("Error: Seed not found in the database.")

    def retrieve_seed(self):
        Institute_id = self.get_institute_id()
        seed_name = self.get_seed_name()

        if Institute_id and seed_name:
            retrieve_date = datetime.now().strftime('%Y-%m-%d')
            retrieve_id = self.get_last_retrieve_id() + 1
            seed_id = self.get_seed_id(seed_name)

            if seed_id:
                # Save retrieve data to the database
                query = f"INSERT INTO RETRIEVAL (RetrievalID, RetrievalDate, INSTITUTE_InstituteID, SEED_SeedID) " \
                        f"VALUES ('{retrieve_id}', STR_TO_DATE('{retrieve_date}', '%Y-%m-%d'), {Institute_id}, {seed_id})"
                self.query_database(query)
                self.result_text.append(f"Retrieve data added for Seed: {seed_name}, Retrieve ID: {retrieve_id}")
            else:
                self.show_error_message("Error: Seed not found in the database.")
        else:
            self.show_error_message("Error: Invalid Institute ID or Seed Name.")

    def deposit_seed(self):
        Institute_id = self.get_institute_id()
        deposit_date = datetime.now().strftime('%Y-%m-%d')
        seed_name = self.get_seed_name()

        if Institute_id and seed_name:
            seed_id = self.get_seed_id(seed_name)

            if seed_id:
                # Get the next Deposit ID
                deposit_id = self.get_next_deposit_id()

                # Save deposit data to the database
                query = f"INSERT INTO DEPOSIT (DepositID, DepositDate, INSTITUTE_InstituteID, SEED_SeedID) " \
                        f"VALUES ({deposit_id}, '{deposit_date}', {Institute_id}, {seed_id})"
                self.query_database(query)
                self.result_text.append(f"Deposit data added for Seed: {seed_name}, Deposit ID: {deposit_id}")
            else:
                self.show_error_message("Error: Seed not found in the database.")
        else:
            self.show_error_message("Error: Invalid Institute ID or Seed Name.")

    def show_insert_seed_dialog(self):
        full_scientific_name, ok = QInputDialog.getText(self, 'Insert Seed Info', 'Enter FullScientificName:')
        if ok and full_scientific_name:
            self.insert_seed(full_scientific_name)

    def insert_seed(self, full_scientific_name):
        query = f"INSERT INTO SEED (FullScientificName) VALUES ('{full_scientific_name}');"
        result = self.query_database(query)

    def get_next_deposit_id(self):
        next_deposit_id = self.query_database("SELECT MAX(DepositID) + 1 FROM DEPOSIT")[0][0]
        return next_deposit_id if next_deposit_id else 1

    def get_institute_id(self):
        query = f"SELECT InstituteID FROM INSTITUTE WHERE InstituteCode = '{self.institute_code}'"
        result = self.query_database(query)

        if result:
            return result[0][0]
        else:
            self.show_error_message("Error: Institute not found in the database.")
            return None

    def get_seed_name(self):
        return self.seed_name_input.text()

    def get_seed_id(self, seed_name):
        seed_id = self.query_database(f"SELECT SeedID FROM SEED WHERE SeedName = '{seed_name}'")
        if seed_id:
            return seed_id[0][0]
        else:
            return None

    def get_last_retrieve_id(self):
        last_retrieve_id = self.query_database("SELECT MAX(RetrievalID) FROM RETRIEVAL")[0][0]
        return last_retrieve_id if last_retrieve_id else 0
    
    def show_view_species_dialog(self):
        species_name, ok = QInputDialog.getText(self, 'View Species Holder', 'Enter FullScientificName:')
        if ok and species_name:
            result = self.view_species_holder(species_name)
            self.show_result_dialog(result)

    def view_species_holder(self, species_name):
        query = f"CALL VIEW_SPECIES_HOLDER('{species_name}')"
        result = self.query_database(query)
        self.display_result(result)

    def query_database(self, query):
        try:
            connection = mysql.connector.connect(
                host="127.0.0.1",
                user="root",
                password="ch141ang",
                database="mydb"
            )
            cursor = connection.cursor()
            cursor.execute(query)
            result = cursor.fetchall()
            connection.next_result()  # Move to the next result set
            connection.commit()
            connection.close()
            return result
        except mysql.connector.Error as err:
            print(f"Database Error: {err}")
            self.show_error_message(f"Database Error: {err}")
            return None

    def display_result(self, result):
        self.result_text.clear()
        if result:
            for row in result[0]:  # Assuming only one row is returned
                self.result_text.append(f"{row}\n")
        else:
            self.result_text.append(f"No data found for the specified Seed Name.")

    def show_error_message(self, message):
        error_dialog = QMessageBox(self)
        error_dialog.setIcon(QMessageBox.Critical)
        error_dialog.setText(message)
        error_dialog.setWindowTitle("Error")
        error_dialog.exec_()

    def test_database_connection(self):
        try:
            connection = mysql.connector.connect(
                host="127.0.0.1",
                user="root",
                password="ch141ang",
                database="mydb"
            )
            connection.close()
            return True
        except mysql.connector.Error as err:
            print(f"Database Connection Error: {err}")
            return False

class LoginDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.init_ui()

    def init_ui(self):
        self.setWindowTitle('Login')
        self.setGeometry(400, 400, 300, 150)

        self.label_Institute_code = QLabel('Institute Code:', self)
        self.Institute_code_input = QLineEdit(self)

        button_box = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel)
        button_box.accepted.connect(self.accept)
        button_box.rejected.connect(self.reject)

        layout = QVBoxLayout()
        layout.addWidget(self.label_Institute_code)
        layout.addWidget(self.Institute_code_input)
        layout.addWidget(button_box)

        self.setLayout(layout)

    def accept(self):
        Institute_code = self.Institute_code_input.text()

        # Validate the Institute code against the database
        if self.validate_Institute_code(Institute_code):
            super().accept()
        else:
            self.show_error_message("Error: Invalid Institute Code.")

    def validate_Institute_code(self, Institute_code):
        query = f"SELECT COUNT(*) FROM INSTITUTE WHERE InstituteCode = '{Institute_code}'"

        try:
            connection = mysql.connector.connect(
                host="127.0.0.1",
                user="root",
                password="ch141ang",
                database="mydb"
            )
            cursor = connection.cursor()
            cursor.execute(query)
            result = cursor.fetchone()[0]
            connection.close()

            return result > 0
        except mysql.connector.Error as err:
            print(f"Database Error: {err}")
            self.show_error_message(f"Database Error: {err}")
            return False

    def get_institute_code(self):
        return self.Institute_code_input.text()

    def show_error_message(self, message):
        error_dialog = QMessageBox(self)
        error_dialog.setIcon(QMessageBox.Critical)
        error_dialog.setText(message)
        error_dialog.setWindowTitle("Error")
        error_dialog.exec_()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    seed_app = SeedApp()
    sys.exit(app.exec_())


KeyboardInterrupt: 